In [4]:
import re
import torch
from PIL import Image
import gc
import requests
import copy
from PIL import Image
from io import BytesIO
from torch.nn.functional import mse_loss
import numpy as np
import einops
from llava.model.builder import load_pretrained_model
from llava.mm_utils import get_model_name_from_path
from llava.eval.run_llava import eval_model
from llava.conversation import conv_templates
from llava.constants import (
    IMAGE_TOKEN_INDEX,
    DEFAULT_IMAGE_TOKEN,
    DEFAULT_IM_START_TOKEN,
    DEFAULT_IM_END_TOKEN,
    IMAGE_PLACEHOLDER,
)
from llava.mm_utils import (
    process_images,
    tokenizer_image_token,
    get_model_name_from_path,
)

from utils.models.factory import create_model_and_transforms, get_tokenizer
from utils.models.prs_hook import hook_prs_logger


In [5]:
# Helper functions
def image_parser(image_file, sep=","):
    out = image_file.split(sep)
    return out


def load_image(image_file):
    if image_file.startswith("http") or image_file.startswith("https"):
        response = requests.get(image_file)
        image = Image.open(BytesIO(response.content)).convert("RGB")
    else:
        image = Image.open(image_file).convert("RGB")
    return image


def load_images(image_files):
    out = []
    for image_file in image_files:
        image = load_image(image_file)
        out.append(image)
    return out

device = "cuda"
model_name = "liuhaotian/llava-v1.5-7b"
model_path = "/cluster/work/vogtlab/Group/vstrozzi/cache/models--liuhaotian--llava-v1.5-7b/snapshots/4481d270cc22fd5c4d1bb5df129622006ccd9234/"

## Get LLava Model

In [6]:
### IMPORT: On Biomedcluster change .config under model_path to point towards correct vision_tower clip path
tokenizer, model, image_processor, context_len = load_pretrained_model(
    model_path=model_path,
    model_base=None,
    model_name=get_model_name_from_path(model_name)
)

vision_enc = model.get_vision_tower()

print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in vision_enc.parameters()]):,}")

You are using a model of type llava to instantiate a model of type llava_llama. This is not supported for all configurations of models and can yield errors.


LlavaConfig {
  "_name_or_path": "/cluster/work/vogtlab/Group/vstrozzi/cache/models--liuhaotian--llava-v1.5-7b/snapshots/4481d270cc22fd5c4d1bb5df129622006ccd9234/",
  "architectures": [
    "LlavaLlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "freeze_mm_mlp_adapter": false,
  "freeze_mm_vision_resampler": false,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "image_aspect_ratio": "pad",
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_length": 4096,
  "max_position_embeddings": 4096,
  "mm_hidden_size": 1024,
  "mm_projector_type": "mlp2x_gelu",
  "mm_resampler_type": null,
  "mm_use_im_patch_token": false,
  "mm_use_im_start_end": false,
  "mm_vision_select_feature": "patch",
  "mm_vision_select_layer": -2,
  "mm_vision_tower": "/cluster/work/vogtlab/Group/vstrozzi/cache/models--openai--clip-vit-large-patch14-336/snapshots/ce19dc912ca5cd21c8a653c79e251e808ccabcd1/",
  "model_type": "llava_lla

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]/cluster/apps/vogtlab/users/vstrozzi/software/anaconda/envs/MT/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Loading checkpoint shards: 100%|██████████| 2/2 [01:29<00:00, 44.71s/it]


Model parameters: 303,507,456


In [12]:
print(vision_enc.vision_tower)
print(vision_enc.vision_tower.vision_model.encoder.__dict__)

CLIPVisionModel(
  (vision_model): CLIPVisionTransformer(
    (embeddings): CLIPVisionEmbeddings(
      (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
      (position_embedding): Embedding(577, 1024)
    )
    (pre_layrnorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-23): 24 x CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=1024, out_features=4096, bias=

NameError: name 'torchview' is not defined

In [16]:
from torchviz import make_dot

# Create a dummy input that matches your model's expected input shape
dummy_input = torch.randn(1, 3, 336, 336)  # adjust shape as needed

# Pass the dummy input through your model
output = model1(dummy_input)

# Create the graph visualization
dot = make_dot(output, params=dict(model1.named_parameters()))

# Render the graph to a file (e.g., PNG)
dot.render("model1_visualization", format="png")

output = model2(dummy_input)
# Create the graph visualization
dot = make_dot(output, params=dict(model1.named_parameters()))

# Render the graph to a file (e.g., PNG)
dot.render("model1_visualization", format="png")




('vision_tower.vision_model.embeddings.class_embedding', Parameter containing:
tensor([ 0.0137,  0.2371, -0.1284,  ...,  0.0171, -0.3352, -0.2383],
       device='cuda:0'))
('vision_tower.vision_model.embeddings.patch_embedding.weight', Parameter containing:
tensor([[[[ 2.4582e-02,  1.0330e-02,  7.1983e-03,  ...,  2.3529e-02,
            2.1957e-02,  5.0011e-03],
          [ 1.2756e-02, -6.1417e-03, -4.8676e-03,  ...,  1.6586e-02,
            7.5417e-03, -1.2230e-02],
          [ 9.9945e-03,  2.1954e-03,  2.3632e-03,  ...,  6.1607e-03,
            5.3940e-03, -1.2283e-02],
          ...,
          [-9.9182e-03, -5.7507e-04, -5.5237e-03,  ..., -1.8646e-02,
           -2.3239e-02, -2.4017e-02],
          [-3.8624e-03, -1.1452e-02, -1.3794e-02,  ..., -7.1030e-03,
            1.5745e-03, -4.1771e-03],
          [-2.1545e-02, -4.2786e-02, -2.9480e-02,  ..., -4.9706e-03,
            4.0169e-03, -6.7177e-03]],

         [[ 1.4839e-02, -5.3024e-03, -1.2367e-02,  ...,  2.5696e-02,
            2

ExecutableNotFound: failed to execute PosixPath('dot'), make sure the Graphviz executables are on your systems' PATH

In [8]:
# CLIP model
model_CLIP_name = 'ViT-L-14-336' 
pretrained = "hf"
precision = "fp16"
model_CLIP, _, preprocess_clip = create_model_and_transforms(model_CLIP_name, pretrained=pretrained, precision=precision, cache_dir="../cache")

model_CLIP.to(device)
model_CLIP.eval()
context_length = model_CLIP.context_length
# Not needed anymore
vocab_size = model_CLIP.vocab_size
# tokenizer_CLIP = get_tokenizer(model_CLIP_name)

print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in model_CLIP.visual.parameters()]):,}")
print("Context length:", context_length)
print("Vocab size:", vocab_size)
print("Len of res:", len(model_CLIP.visual.transformer.resblocks))
# Replace with the one we want to use
model2 = model_CLIP.visual
model1 = vision_enc
# prs = hook_prs_logger(model_CLIP, device, spatial=True, vision_projection=False) # This attach hook to get the residual stream
print(model_CLIP.visual)

Using local files
Model parameters: 304,293,888
Context length: 77
Vocab size: 49408
Len of res: 24
VisionTransformer(
  (patchnorm_pre_ln): Identity()
  (conv1): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
  (patch_dropout): Identity()
  (ln_pre): LayerNorm()
  (transformer): Transformer(
    (resblocks): ModuleList(
      (0-23): 24 x ResidualAttentionBlock(
        (ln_1): LayerNorm()
        (attn): MultiheadAttention(
          (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (ls_1): Identity()
        (ln_2): LayerNorm()
        (mlp): MLP(
          (c_fc): Linear(in_features=1024, out_features=4096, bias=True)
          (gelu): GELU(approximate='none')
          (c_proj): Linear(in_features=4096, out_features=1024, bias=True)
        )
        (ls_2): Identity()
      )
    )
  )
  (ln_post): LayerNorm()
)


In [6]:
for var_name in model1.state_dict():
    print(var_name, "\t", model1.state_dict()[var_name].shape, "\t", model1.state_dict()[var_name])
    print()

vision_tower.vision_model.embeddings.class_embedding 	 torch.Size([1024]) 	 tensor([ 0.0137,  0.2371, -0.1284,  ...,  0.0171, -0.3352, -0.2383],
       device='cuda:0')

vision_tower.vision_model.embeddings.patch_embedding.weight 	 torch.Size([1024, 3, 14, 14]) 	 tensor([[[[ 2.4582e-02,  1.0330e-02,  7.1983e-03,  ...,  2.3529e-02,
            2.1957e-02,  5.0011e-03],
          [ 1.2756e-02, -6.1417e-03, -4.8676e-03,  ...,  1.6586e-02,
            7.5417e-03, -1.2230e-02],
          [ 9.9945e-03,  2.1954e-03,  2.3632e-03,  ...,  6.1607e-03,
            5.3940e-03, -1.2283e-02],
          ...,
          [-9.9182e-03, -5.7507e-04, -5.5237e-03,  ..., -1.8646e-02,
           -2.3239e-02, -2.4017e-02],
          [-3.8624e-03, -1.1452e-02, -1.3794e-02,  ..., -7.1030e-03,
            1.5745e-03, -4.1771e-03],
          [-2.1545e-02, -4.2786e-02, -2.9480e-02,  ..., -4.9706e-03,
            4.0169e-03, -6.7177e-03]],

         [[ 1.4839e-02, -5.3024e-03, -1.2367e-02,  ...,  2.5696e-02,
        

In [7]:
for var_name in model2.state_dict():
    print(var_name, "\t", model2.state_dict()[var_name].shape, "\t", model2.state_dict()[var_name])

class_embedding 	 torch.Size([1024]) 	 tensor([ 0.0137,  0.2371, -0.1284,  ...,  0.0171, -0.3352, -0.2383],
       device='cuda:0', dtype=torch.float16)
positional_embedding 	 torch.Size([577, 1024]) 	 tensor([[ 0.0017,  0.0484, -0.0145,  ...,  0.0005, -0.0566, -0.0468],
        [ 0.0112, -0.0406,  0.0338,  ...,  0.0249, -0.0308, -0.0376],
        [ 0.0016, -0.0357,  0.0118,  ...,  0.0215, -0.0292, -0.0399],
        ...,
        [-0.0033, -0.0346, -0.0031,  ..., -0.0287, -0.0336, -0.0360],
        [-0.0055, -0.0331, -0.0074,  ..., -0.0286, -0.0326, -0.0354],
        [-0.0067, -0.0280, -0.0148,  ..., -0.0182, -0.0268, -0.0353]],
       device='cuda:0', dtype=torch.float16)
proj 	 torch.Size([1024, 768]) 	 tensor([[ 0.0220, -0.0138, -0.0074,  ..., -0.0050, -0.0073,  0.0134],
        [ 0.0184,  0.0082,  0.0390,  ..., -0.0151, -0.0242, -0.0007],
        [ 0.0072, -0.0003,  0.0196,  ..., -0.0059, -0.0077,  0.0157],
        ...,
        [ 0.0118, -0.0161, -0.0146,  ..., -0.0063,  0.0089,  0.

In [8]:
model1.eval()
model2.eval()
for param in model2.parameters():
    param.requires_grad = False
print(model2.ln_pre.__dict__)
print(model1.vision_tower.vision_model.pre_layrnorm.__dict__)

{'training': False, '_parameters': OrderedDict([('weight', Parameter containing:
tensor([0.3305, 0.0026, 0.1605,  ..., 2.1931, 0.0041, 0.0044], device='cuda:0')), ('bias', Parameter containing:
tensor([-0.0044, -0.0450, -0.0473,  ...,  0.0395, -0.1389, -0.0134],
       device='cuda:0'))]), '_buffers': OrderedDict(), '_non_persistent_buffers_set': set(), '_backward_pre_hooks': OrderedDict(), '_backward_hooks': OrderedDict(), '_is_full_backward_hook': None, '_forward_hooks': OrderedDict(), '_forward_hooks_with_kwargs': OrderedDict(), '_forward_hooks_always_called': OrderedDict(), '_forward_pre_hooks': OrderedDict(), '_forward_pre_hooks_with_kwargs': OrderedDict(), '_state_dict_hooks': OrderedDict(), '_state_dict_pre_hooks': OrderedDict(), '_load_state_dict_pre_hooks': OrderedDict(), '_load_state_dict_post_hooks': OrderedDict(), '_modules': OrderedDict(), 'hook': <utils.models.hook.HookManager object at 0x148d4e27c3a0>, 'normalized_shape': (1024,), 'eps': 1e-05, 'elementwise_affine': True

In [9]:
model1.state_dict().keys()
model2.state_dict().keys()

odict_keys(['class_embedding', 'positional_embedding', 'proj', 'conv1.weight', 'ln_pre.weight', 'ln_pre.bias', 'transformer.resblocks.0.ln_1.weight', 'transformer.resblocks.0.ln_1.bias', 'transformer.resblocks.0.attn.in_proj_weight', 'transformer.resblocks.0.attn.in_proj_bias', 'transformer.resblocks.0.attn.out_proj.weight', 'transformer.resblocks.0.attn.out_proj.bias', 'transformer.resblocks.0.ln_2.weight', 'transformer.resblocks.0.ln_2.bias', 'transformer.resblocks.0.mlp.c_fc.weight', 'transformer.resblocks.0.mlp.c_fc.bias', 'transformer.resblocks.0.mlp.c_proj.weight', 'transformer.resblocks.0.mlp.c_proj.bias', 'transformer.resblocks.1.ln_1.weight', 'transformer.resblocks.1.ln_1.bias', 'transformer.resblocks.1.attn.in_proj_weight', 'transformer.resblocks.1.attn.in_proj_bias', 'transformer.resblocks.1.attn.out_proj.weight', 'transformer.resblocks.1.attn.out_proj.bias', 'transformer.resblocks.1.ln_2.weight', 'transformer.resblocks.1.ln_2.bias', 'transformer.resblocks.1.mlp.c_fc.weight'

In [10]:
import torch
import torch.nn as nn

# Make sure both models are in evaluation mode to avoid randomness from dropout/batchnorm.
model1.eval()
model2.eval()

model1 = model1
model2 = model2
# Dictionaries to store activations from each model.
activations_model1 = {}
activations_model2 = {}

# ---------------------------------------------------------------------
# Helper function to create a hook that saves the module's output.
def get_activation(name, activation_dict):
    def hook(module, input, output):
        # If the module returns a tuple, take the first element.
        if isinstance(output, tuple):
            output = output[0]
        # Store a detached copy of the output on CPU (for easy comparison).
        activation_dict[name] = output.detach().cpu()
    return hook

# =============================================
# Register hooks for Model 1 (CLIPVisionTower)
# =============================================
# Patch embedding (note: using the field as defined in your model)
model1_patch_embed = model1.vision_tower.vision_model.embeddings.patch_embedding
model1_patch_embed.register_forward_hook(get_activation("patch_embedding", activations_model1))

# Pre-layernorm (note the field name is "pre_layrnorm")
model1_pre_ln = model1.vision_tower.vision_model.pre_layrnorm
model1_pre_ln.register_forward_hook(get_activation("pre_layrnorm", activations_model1))

# Encoder layers (assuming 24 layers stored in a ModuleList)
encoder_layers_model1 = model1.vision_tower.vision_model.encoder.layers
for i, layer in enumerate(encoder_layers_model1):
    layer.register_forward_hook(get_activation(f"encoder_layer_{i}", activations_model1))

# Post-layernorm
model1_post_ln = model1.vision_tower.vision_model.post_layernorm
model1_post_ln.register_forward_hook(get_activation("postlayernorm", activations_model1))

# =============================================
# Register hooks for Model 2 (VisionTransformer)
# =============================================
# Patch embedding (corresponds to conv1 in Model 2)
model2_patch_embed = model2.conv1
model2_patch_embed.register_forward_hook(get_activation("conv1", activations_model2))

# Pre-layernorm
model2_pre_ln = model2.ln_pre
model2_pre_ln.register_forward_hook(get_activation("ln_pre", activations_model2))

# Transformer blocks (24 ResidualAttentionBlock modules)
resblocks_model2 = model2.transformer.resblocks
for i, block in enumerate(resblocks_model2):
    block.register_forward_hook(get_activation(f"resblock_{i}", activations_model2))

# Post-layernorm
model2_post_ln = model2.ln_post
model2_post_ln.register_forward_hook(get_activation("ln_post", activations_model2))

# =============================================
# Run a forward pass with the same input for both models.
# =============================================
# Choose an input size that is valid for the patch size (e.g., 336x336 in your example).
# Load images from online or local
image_file = "images/catdog.png"
sep =  ","
image_files = image_parser(image_file, sep)
images = load_images(image_files)
image_sizes = [x.size for x in images]
# Convert images to format b, 3, h, w (h = w) with resizing or padding
images_tensor = process_images(
    images,
    image_processor,
    model.config
).to(model.device, dtype=torch.float16)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
x = images_tensor

# Forward pass
output_model1 = model1(x)
output_model2 = model2(x)

# =============================================
# Compare corresponding activations
# =============================================

# High-level mapping between model parts (names are different, so we define a mapping)
layer_mapping = {
    "patch_embedding": "conv1",    # Patch embedding: Model 1 vs. Model 2
    "pre_layrnorm": "ln_pre",        # Pre-layernorm
    "postlayernorm": "ln_post"       # Post-layernorm
}

print("Comparing high-level activations:")
for name1, name2 in layer_mapping.items():
    act1 = activations_model1.get(name1)
    act2 = activations_model2.get(name2)
    if act1 is not None and act2 is not None:
        diff = torch.norm(act1 - act2.to(dtype=torch.float32))
        print(act1.shape)
        print(act2.shape)
        print(torch.norm(act1))
        print(torch.norm(act2))
        print(f"Difference between Model1 '{name1}' and Model2 '{name2}': {diff.item()}")
    else:
        print(f"Missing activation for Model1 '{name1}' or Model2 '{name2}'")

# Compare transformer / encoder layers block-by-block (assuming 24 layers in each)
print("\nComparing transformer/encoder layers:")
for i in range(24):
    act1 = activations_model1.get(f"encoder_layer_{i}")[:, 1:, :]
    act2 = activations_model2.get(f"resblock_{i}")[:, :-1, :]
    if act1 is not None and act2 is not None:
        print(f"resblock_{i}")
        print
        diff = torch.norm(act1 - act2)
        print(torch.norm(act1))
        print(act1.shape)
        print(torch.norm(act2))
        print(act2.shape)
        print(f"Difference at block {i}: {diff.item()}")
    else:
        print(f"Missing activation for block {i}")

# Finally, compare the final outputs of both models
print(output_model1.shape)
print(output_model2.shape)

print(torch.norm(output_model1))
print(torch.norm(output_model2))

final_diff = torch.norm(output_model1 - output_model2.T)
print(f"\nDifference in final outputs: {final_diff.item()}")


Comparing high-level activations:
torch.Size([1, 1024, 24, 24])
torch.Size([1, 1024, 24, 24])
tensor(193.9296)
tensor(193.8750, dtype=torch.float16)
Difference between Model1 'patch_embedding' and Model2 'conv1': 0.04036330431699753
torch.Size([1, 577, 1024])
torch.Size([1, 577, 1024])
tensor(219.6352)
tensor(219.6250, dtype=torch.float16)
Difference between Model1 'pre_layrnorm' and Model2 'ln_pre': 0.11766563355922699
torch.Size([1, 1024])
torch.Size([1, 1024])
tensor(33.3382)
tensor(33.4688, dtype=torch.float16)
Difference between Model1 'postlayernorm' and Model2 'ln_post': 10.44230842590332

Comparing transformer/encoder layers:
resblock_0
tensor(208.4664)
torch.Size([1, 576, 1024])
tensor(210.8750, dtype=torch.float16)
torch.Size([1, 576, 1024])
Difference at block 0: 224.16041564941406
resblock_1
tensor(191.1261)
torch.Size([1, 576, 1024])
tensor(201.6250, dtype=torch.float16)
torch.Size([1, 576, 1024])
Difference at block 1: 201.18299865722656
resblock_2
tensor(201.1244)
torch.

RuntimeError: The size of tensor a (576) must match the size of tensor b (768) at non-singleton dimension 1

In [8]:
print(activations_model1)

{'patch_embedding': tensor([[[[ 2.8754e-02, -9.7862e-02,  1.6143e-01,  ...,  6.1842e-01,
           -1.9362e-01, -1.5486e-03],
          [-8.9286e-02, -6.3885e-01, -2.5598e-01,  ...,  1.4553e-01,
            4.0429e-01,  5.8272e-01],
          [ 1.9795e-01,  1.2640e-01,  3.7088e-01,  ..., -2.5186e-01,
            9.0923e-02,  1.9063e-01],
          ...,
          [ 4.1506e-01,  1.7928e-01, -1.9997e-01,  ..., -4.7764e-01,
            4.7578e-01, -6.2456e-02],
          [ 1.4842e-01, -4.1643e-01, -1.5682e-01,  ...,  8.3002e-01,
            7.0565e-02,  2.7281e-01],
          [-3.9231e-01,  3.5112e-01,  3.7072e-01,  ...,  7.3410e-02,
           -5.9967e-02,  2.0974e-01]],

         [[-9.2655e-01, -2.9789e-01, -2.9725e-01,  ..., -4.5696e-02,
           -2.8924e-01,  2.7398e-01],
          [ 1.5798e-01, -9.8029e-01,  1.2270e+00,  ...,  1.1767e+00,
           -8.2230e-01, -3.1694e-01],
          [-4.2003e-02, -2.2545e-01,  5.2032e-03,  ..., -8.3201e-02,
           -7.9196e-01,  7.6895e-02],


## Here play around with LLava

In [13]:
# Layer where to extract infos on patches
select_layer = -2
# Params
prompt = "Describe the image focusing on main subjects. ignore background"
image_file = "images/catdog.png"
max_new_tokens = 512
num_beams = 1 # numer of path of decision, less faster
sep =  ","
temperature = 0 # 0 lowest, det
top_p = None
images_embeds = True # If provided image embeds

image_token_se = DEFAULT_IM_START_TOKEN + DEFAULT_IMAGE_TOKEN + DEFAULT_IM_END_TOKEN
# Making prompt in correct format
if IMAGE_PLACEHOLDER in prompt:
    if model.config.mm_use_im_start_end:
        prompt = re.sub(IMAGE_PLACEHOLDER, image_token_se, prompt)
    else:
        prompt = re.sub(IMAGE_PLACEHOLDER, DEFAULT_IMAGE_TOKEN, prompt)
else:
    if model.config.mm_use_im_start_end:
        prompt = image_token_se + "\n" + prompt
    else:
        prompt = DEFAULT_IMAGE_TOKEN + "\n" + prompt

# Derive necessary conv
if "llama-2" in model_name.lower():
    conv_mode = "llava_llama_2"
elif "mistral" in model_name.lower():
    conv_mode = "mistral_instruct"
elif "v1.6-34b" in model_name.lower():
    conv_mode = "chatml_direct"
elif "v1" in model_name.lower():
    conv_mode = "llava_v1"
elif "mpt" in model_name.lower():
    conv_mode = "mpt"
else:
    conv_mode = "llava_v0"

if conv_mode is not None and conv_mode != conv_mode:
    print(
        "[WARNING] the auto inferred conversation mode is {}, while `--conv-mode` is {}, using {}".format(
            conv_mode, conv_mode, conv_mode
        )
    )
else:
    conv_mode = conv_mode

# Load conversation mode standard template 
conv = conv_templates[conv_mode].copy()
conv.append_message(conv.roles[0], prompt)
conv.append_message(conv.roles[1], None)
prompt = conv.get_prompt()
print(prompt)

# Load images from online or local
image_files = image_parser(image_file, sep)
images = load_images(image_files)
image_sizes = [x.size for x in images]
# Convert images to format b, 3, h, w (h = w) with resizing or padding
images_tensor = process_images(
    images,
    image_processor,
    model.config
).to(model.device, dtype=torch.float16)


# Tokenize prompt
input_ids = (
    tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors="pt")
    .unsqueeze(0)
    .to(device)
)


# Use CLIP Model
if images_embeds:
    print()
    vision_enc = vision_enc.to(dtype=torch.float16)
    output_pt = vision_enc(images_tensor.to(device))
    print(output_pt.norm())
    print(output_pt.shape)
    #spatial_features = torch.randn((1, 576, 1024))# if want to manually edit features, we want b, nr_spat (576), d (1024)
    # Need to handle some memory movement :/
    model.to("cpu")
    gc.collect()
    torch.cuda.empty_cache()

   # prs.reinit()  # Reinitialize any hooks if required
    with torch.no_grad():

        """ spatial_features = model_CLIP.encode_image(
                images_tensor.to(device), 
                attn_method='head',
                normalize=False
            )

        attentions, mlps = prs.finalize(spatial_features)  # attentions: [1, 12, 197, 16, 512], [b, l, n, h, d], mlps: [1, 13, 512], [b, l + 1, d]
        attentions = einops.rearrange(attentions, "b l n h d -> b l h n d")
        # Compute intermediate features from select_layer
        print(attentions.shape)
        print(mlps.shape)
        for k in range(1, 24 - 2):
            if k == 1:
                spatial_features = attentions[:, k, :, 1:, :].sum(1) + mlps[:, k, 1:, :]
            else:
                spatial_features += spatial_features + attentions[:, k, :, 1:, :].sum(1) + mlps[:, k, 1:, :]

            print(spatial_features.norm())
        #spatial_features = (attentions[:, :-2, :, 1:, :].sum(1).sum(1) + mlps[:, :-1, 1:, :].sum(1))
        print(spatial_features.shape)
        print(spatial_features.norm())
        print("MSE Loss:", mse_loss(output_pt.to("cpu"), spatial_features.to("cpu")).item())

        print(spatial_features.norm())
        print(attentions.shape, mlps.shape) """
        
    # Move model back to gpu
    model.to("cuda")
    images_tensor = ct2 = output_pt#activations_model2.get(f"resblock_{23}")[:, 1:, :].to(device)
    print(images_tensor.shape)


# Generate an answer by using full model LLava
with torch.inference_mode():
    output_ids = model.generate(
        input_ids,
        images=images_tensor,
        image_sizes=image_sizes,
        do_sample=True if temperature > 0 else False,
        temperature=temperature,
        top_p=top_p,
        num_beams=num_beams,
        max_new_tokens=max_new_tokens,
        use_cache=True,
        images_embeds = images_embeds # If want to give images embeds already precomputed TODO: Only support 1 image

    )

# Print the output
outputs = tokenizer.batch_decode(output_ids, skip_special_tokens=True)[0].strip()
print(outputs)

A chat between a curious human and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the human's questions. USER: <image>
Describe the image focusing on main subjects. ignore background ASSISTANT:

tensor(734.5000, device='cuda:0', dtype=torch.float16)
torch.Size([1, 576, 1024])
torch.Size([1, 576, 1024])


/cluster/apps/vogtlab/users/vstrozzi/software/anaconda/envs/MT/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/cluster/apps/vogtlab/users/vstrozzi/software/anaconda/envs/MT/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `None` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


The image features a brown and white cat lying on the floor next to a brown and white dog. The cat is positioned on the left side of the dog, creating a sense of companionship between the two animals. The dog appears to be sitting or standing, while the cat is relaxed on the floor. The scene takes place in a room with a window, providing natural light to the space.
